<a href="https://colab.research.google.com/github/Nguyen-Kim-Son/large-language-models/blob/main/DeepSeek_R1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [7]:
MODEL = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit"

In [8]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
)


==((====))==  Unsloth 2025.2.5: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.77k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.2.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [11]:
from datasets import load_dataset


In [12]:
prompt_style = """### Instruction:
You are an expert Python programmer. Please solve the following coding problem.

### Question:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    return {"text": [prompt_style.format(p, c) + EOS_TOKEN
                     for p, c in zip(examples["prompt"], examples["completion"])]}

dataset = load_dataset("sdiazlor/python-reasoning-dataset", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)


README.md:   0%|          | 0.00/6.59k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/844k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [14]:
from trl import SFTTrainer


In [35]:
print(dataset.filter(lambda x: x["prompt"] is None or x["completion"] is None))


Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'completion', 'system_prompt'],
    num_rows: 1
})


In [36]:
dataset = dataset.filter(lambda x: x["prompt"] is not None and x["completion"] is not None)


Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [37]:
def clean_data(examples):
    return {
        "prompt": [p if p is not None else "" for p in examples["prompt"]],
        "completion": [c if c is not None else "" for c in examples["completion"]],
    }

dataset = dataset.map(clean_data, batched=True)


Map:   0%|          | 0/499 [00:00<?, ? examples/s]

In [40]:
def convert_to_text(examples):
    return {"text": [str(p) + " " + str(c) for p, c in zip(examples["prompt"], examples["completion"])]}

dataset = dataset.map(convert_to_text, batched=True)
print(dataset.column_names)  # Kiểm tra xem đã có "text" chưa


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

['prompt', 'completion', 'system_prompt', 'text']


In [42]:
print(train_dataset.column_names)  # Phải chứa "text"
print(eval_dataset.column_names)   # Phải chứa "text"


['prompt', 'completion', 'system_prompt']
['prompt', 'completion', 'system_prompt']


In [52]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("sdiazlor/python-reasoning-dataset", split="train")

# Chia dữ liệu thành train (90%) và eval (10%)
split_dataset = dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# Chuyển đổi sang dạng có cột "text"
def convert_to_text(examples):
    return {"text": [str(p) + " " + str(c) for p, c in zip(examples["prompt"], examples["completion"])]}

train_dataset = train_dataset.map(convert_to_text, batched=True)
eval_dataset = eval_dataset.map(convert_to_text, batched=True)

# Kiểm tra lại
print(train_dataset.column_names)  # Phải có "text"
print(eval_dataset.column_names)   # Phải có "text"


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

['prompt', 'completion', 'system_prompt', 'text']
['prompt', 'completion', 'system_prompt', 'text']


In [53]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False,
    args=training_arguments,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/450 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

In [54]:
model.save_pretrained_merged("fine_tuned_model", tokenizer, save_method="merged_16bit")


Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 1.8G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.67 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 18.22it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving fine_tuned_model/pytorch_model.bin...
Done.


In [55]:
question = "How can I get the prime numbers from 0 to 125?"

FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=2048,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)
print(response)


["<｜begin▁of▁sentence｜>### Instruction:\nYou are an expert Python programmer. Please solve the following coding problem.\n\n### Question:\nHow can I get the prime numbers from 0 to 125?\n\n### Response:\n\nTo get the prime numbers from 0 to 125, I need to follow a systematic approach. First, I'll create a list of all numbers in that range. Then, I'll iterate through each number and check if it's a prime number.\n\nTo check if a number is prime, I'll need a helper function. This function will take a number and return True if it's prime, False otherwise. The helper function will perform the following steps:\n\n1. If the number is less than 2, it's not prime.\n2. If the number is 2, it's prime.\n3. If the number is even, it's not prime.\n4. Then, I'll check divisibility from 3 up to the square root of the number, stepping by 2. If any of these numbers divide the number, it's not prime.\n\nOnce I've determined if a number is prime, I'll add it to the list of primes. Finally, I'll return th